In [32]:
import cvxpy as cp
import numpy as np

In [33]:
returns = np.load('./data/returns.npy')
stocks = np.array(open('./data/names_s&p.txt').read().splitlines())

mean_returns = np.mean(returns, axis=1)
A = (returns - mean_returns[:, np.newaxis]).T

In [34]:
m = returns.shape[0] # Number of assets
T = returns.shape[1] # Number of time periods
B = 100000 # Budget
min_return = 0.03
U = 10 # Allowed number of distinct assets

In [35]:
def solve_problem(U):
    # Define variables
    x = cp.Variable(m)
    y = cp.Variable(m, boolean=True)
    z = cp.Variable(T)

    # Define objective function
    objective = cp.Minimize(1/T * cp.sum(z))

    # Form constraints
    constraints = []
    constraints.append(z - A @ x >= 0)  # From absolute value
    constraints.append(z + A @ x >= 0)  # From absolute value
    constraints.append(cp.sum(x) <= B)  # Budget constraint
    constraints.append(x >= 0)  # No shorting
    constraints.append(cp.sum(cp.multiply(mean_returns, x)) >=
                    min_return * B)  # Minimum return constraint
    constraints.append(cp.sum(y) <= U)  # Number of distinct assets constraint
    constraints.append(x <= B * y)  # If asset is not bought, it's weight is 0

    # Solve problem
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.MOSEK, verbose=False) # Change to True to see solver output

    print("U: ", U)
    print("Optimal value: ", problem.value)

    # Print which indices of x are non-zero
    nnz_idx = np.nonzero(x.value)[0]
    nnz_alloc = x.value[nnz_idx]
    print("Indices of non-zero allocations: ", nnz_idx)
    print("Non-zero allocations: ", nnz_alloc)
    print("Names of companies with non-zero allocations: ", stocks[nnz_idx])

In [36]:
for U in range(5, 11):
    solve_problem(U)

U:  5
Optimal value:  486.6123329028528
Indices of non-zero allocations:  [ 36 168 181 308 311 371]
Non-zero allocations:  [2.58493941e-26 1.03155915e+04 1.70137195e+04 3.13594873e+04
 1.38800296e+04 1.37861331e+04]
Names of companies with non-zero allocations:  ['Ansys' 'Eli Lilly and Company' 'Everest Re' 'McKesson' 'Meta Platforms'
 'Phillips 66']
U:  6
Optimal value:  361.59362611380675
Indices of non-zero allocations:  [ 18 168 241 264 311 371]
Non-zero allocations:  [18764.42904019 13145.00226623 14571.9226078  14552.45709258
 12611.88036759 10611.35084305]
Names of companies with non-zero allocations:  ['Allstate' 'Eli Lilly and Company' 'Humana' 'Jabil' 'Meta Platforms'
 'Phillips 66']
U:  7
Optimal value:  280.75962011480533
Indices of non-zero allocations:  [ 18 168 241 264 311 360 371]
Non-zero allocations:  [17590.16128739 13846.22432999 18342.06167738 14871.81035348
 11101.67099834  3508.29969253  6079.03585074]
Names of companies with non-zero allocations:  ['Allstate' 'E